<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/master/Train2vec_%3E_Matrix_Factorization_%3E_Train2vec_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

train data 전처리 과정

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from collections import Counter
from typing import *
from khaiii import KhaiiiApi
from collections import Counter


In [ ]:
folder = '/content/gdrive/My Drive/melon'
filelist = os.chdir(folder)

In [ ]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
song_meta=pd.read_json('song_meta.json')

In [ ]:
!git clone https://github.com/slipaway/melonplaylist.git

In [ ]:
genre_gn_all.head()

In [ ]:
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

In [ ]:
gnr_code=genre_gn_all[genre_gn_all['gnr_code'].str[-2:]=='00']

In [ ]:
#대장르 장르코드 리스트
gnr_code

In [ ]:
#곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출

song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

#unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))),
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

#unnested 데이터프레임 생성 : song_gnr_map

song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns=song_gnr_map.columns)
song_gnr_map['id']=song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns={'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace=True)

#unnest 객체 제거
del song_gnr_map_unnest

In [ ]:
plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest

In [ ]:
plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)

In [ ]:
strange_songs

In [ ]:
strange_songs=strange_songs.drop_duplicates(['id'])
len(strange_songs)

In [ ]:
strange_songs_list=strange_songs['id'].values.astype(str)
strange_song_gnr_map=song_gnr_map[song_gnr_map['song_id'].isin(strange_songs_list)]

In [ ]:
type(strange_songs_list)

In [ ]:
song_gnr_map

In [ ]:
strange_song_gnr_map

In [ ]:
strange_cnt=strange_song_gnr_map.groupby('gnr_code').count()
strange_cnt=pd.merge(strange_cnt, gnr_code, on='gnr_code')

In [ ]:
strange_cnt

In [ ]:
strange_songs=strange_songs.drop_duplicates(['id'])
print(strange_songs)

In [ ]:
strange_id=strange_songs.loc[:, 'id']
strange_id
strange_id_list=list(strange_id)

In [ ]:
train_id=train.loc[:, 'id']

In [ ]:
song_meta_id=song_meta.loc[:, 'id']
print(song_meta_id.head())

In [ ]:
strange_songs

In [ ]:
#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map=train[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

In [ ]:
#플레이리스트에 수록된 곡의 총 개수(중복 포함)

len(plylst_song_map)

In [ ]:
#플레이리스트에 strange 곡 제거한 후, 수록된 곡의 총 개수(중복 포함)

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))]
plylst_song_map_remove.shape

In [ ]:
plylst_song_map_remove

In [ ]:
#train set에 새로운 column : remove_songs 생성
#remove_songs : strange 곡 제거한 후 다시 만든 플레이리스트

plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
train['id']=train['id'].astype(str)
train_remove = pd.merge(train, plylst_song_group)

train_remove.head()

train data에서 tag와 playlist title을 형태소 분석

In [ ]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list


In [ ]:
tokenizer = KhaiiiApi()
all_tag = get_all_tags(train)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

In [ ]:
token_tag[:10]

In [ ]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)

In [ ]:
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')

In [ ]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()

In [ ]:
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()

In [ ]:
#플레이리스트 제목 형태소 분석

train['plylst_title']=re_sub(train['plylst_title'])
train.loc[:, 'ply_token']=train['plylst_title'].map(lambda x: get_token(x, tokenizer))

In [ ]:
using_pos=['NNG', 'SL', 'NNP', 'MAG', 'SN'] #일반 명사, 외국어, 고유 명사, 일반 부사, 숫자
train['ply_token']=train['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))


In [ ]:
unique_tag=set(token_itself)
unique_word=[x[0] for x in unique_tag]

#train 데이터의 plylst title 형태소 분리

In [ ]:
#정답 tag에 나온 형태소만 남겨두기
train['ply_token']=train['ply_token'].map(lambda x: list(filter(lambda x: x[0] in unique_word, x)))
train.head(10)

형태소 분석한 tag, playlist title과 날짜가 맞지 않는 노래들을 제외한 목록이 추가된 train data

In [ ]:
train_eda = pd.merge(train, plylst_song_group)
train_eda

word2vec 학습을 위해 사용할 데이터들만 모아 놓은 dataframe 생성

In [ ]:
wv_sent = train_eda[['ply_token', 'tags', 'remove_songs']]
wv_sent

플레이리스트 제목을 형태소 분해한 데이터에서 품사정보를 제외

In [ ]:
#Word2vec 이용을 위해 ply_token data를 전처리
z = []
y = []
for i in range(len(wv_sent['ply_token'])):
  z = []
  for j in range(len(wv_sent['ply_token'][i])):
    x = wv_sent['ply_token'][i][j]
    x = list(x)
    del x[1]
    z.insert(j,"".join(x))
  y.insert(i,z)
y

In [ ]:
wv_sent['ply_token_new'] = y
wv_sent = wv_sent[['ply_token_new', 'tags', 'remove_songs']]
wv_sent # Word2vec 이용을 위해 필요한 data들만 모아둔 dataframe

train data에 존재하는 플레이리스트 제목을 형태소 분해한 것들의 리스트
& train data에 존재하는 태그들을 형태소 분해한 것들의 리스트
& train data에 존재하는 노래들의 리스트

In [ ]:
train_token_list = np.dstack(
    (
        np.concatenate(wv_sent.ply_token_new.values)
    )
)
train_token_list = pd.DataFrame(train_token_list[0][0], columns = ['token'])
train_token_list = list(train_token_list['token']) # train data에 존재하는 모든 플레이리스트 제목 형태소들의 list

train_tag_list = np.dstack(
    (
        np.concatenate(wv_sent.tags.values)
    )
)
train_tag_list = pd.DataFrame(train_tag_list[0][0], columns = ['tag'])
train_tag_list = list(train_tag_list['tag']) # train data에 존재하는 모든 태그들의 list

train_song_list = np.dstack(
    (
        np.concatenate(wv_sent.remove_songs.values)
    )
)
train_song_list = pd.DataFrame(train_song_list[0][0], columns = ['song'])
train_song_list = list(train_song_list['song']) # train data에 존재하는 노래들의 list

In [ ]:
train_token_list_unduplicated = list(set(train_token_list))

In [ ]:
train_tag_list_unduplicated = list(set(train_tag_list))


In [ ]:
train_song_list_unduplicated = list(set(train_song_list))


Word2vec 사용을 위해 플레이리스트 별로 제목의 형태소와 태그, 곡 id를 한 문장으로 갖도록 전처리

In [ ]:
#Word2vec 학습을 위해 플레이리스트 별로 제목의 형태소와 태그, 곡 id를 한 문장으로 갖도록 전처리
w = []
for i in range(len(wv_sent)):
  w.insert(i,wv_sent['ply_token_new'][i] + wv_sent['tags'][i] + wv_sent['remove_songs'][i])
train_eda_fin = w
train_eda_fin

word2vec 적용

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
class EpochSaver(CallbackAny2Vec):
     '''Callback to save model after each epoch.'''

     def __init__(self, path_prefix):
         self.path_prefix = path_prefix
         self.epoch = 0

     def on_epoch_end(self, model):
         output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
         model.save(output_path)
         self.epoch += 1

class EpochLogger(CallbackAny2Vec):
     def __init__(self):
         self.epoch = 0

     def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

     def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1


In [ ]:
from gensim.models import Word2Vec
epoch_logger = EpochLogger()
wv_model = Word2Vec(train_eda_fin, window = 10, min_count=7, workers=6, size = 400, iter=50, sg=1, callbacks=[epoch_logger])


In [ ]:
learning_list = wv_model.wv.vocab.keys()
# word2vec에 의해 학습된 것들(플레이 리스트 제목이 형태소로 분해된 것들 & 태그 & 곡 id)의 리스트

test data 불러오기

In [ ]:
test=pd.read_json('test.json')
test

test data 중에서 일부 곡과 일부 태그가 존재하는 경우를 제외한 data set 생성

In [ ]:
me = []
for i in range(len(test['id'])):
  if len(test['plylst_title'][i]) == 0 and len(test['tags'][i]) != 0 and len(test['songs'][i]) !=0:
    me.append(i)
  else:
    pass


In [ ]:
test = test.drop(me, axis = 0)
test = test.reset_index(drop=True)
test # 곡과 태그만 존재하는 플레이리스트를 제외한 나머지 플레이리스트

위의 결과로 얻은 데이터셋 중에서 곡만 존재하는 플레이리스트를 제외한 데이터셋 생성

In [ ]:
mel = []
for i in range(len(test['id'])):
  if len(test['plylst_title'][i]) == 0 and len(test['tags'][i]) == 0 and len(test['songs'][i]) !=0:
    mel.append(i)
  else:
    pass


In [ ]:
test_1 = test.drop(mel, axis = 0)
test_1 = test_1.reset_index(drop=True)
test_1 # 곡과 태그만 존재하는 플레이리스트를 제외한 나머지 플레이리스트 중에서 곡만 존재하는 플레이리스트를 제외 -> 제목+태그 or 제목만 있는 경우

곡과 태그만 존재하는 플레이리스트를 제외한 나머지 플레이리스트 중에서 곡만 존재하는 플레이리스트만 존재하는 데이터셋 생성

In [ ]:
test_2 = test.loc[mel]
test_2 = test_2.reset_index(drop=True)
test_2 # 곡과 태그만 존재하는 플레이리스트를 제외한 나머지 플레이리스트 중에서 곡만 존재하는 플레이리스트

test_1 data의 제목과 태그에 대해 형태소 분석

In [ ]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list

tokenizer = KhaiiiApi()
all_tag = get_all_tags(test_1)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

token_tag[:10]


In [ ]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')


In [ ]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()


In [ ]:
#플레이리스트 제목 형태소 분석
test_1['plylst_title']=re_sub(test_1['plylst_title'])
test_1.loc[:, 'ply_token']=test_1['plylst_title'].map(lambda x: get_token(x, tokenizer))
using_pos=['NNG', 'SL', 'NNP', 'MAG', 'SN'] #일반 명사, 외국어, 고유 명사, 일반 부사, 숫자
test_1['ply_token']=test_1['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
unique_tag=set(token_itself)
unique_word=[x[0] for x in unique_tag]
#정답 tag에 나온 형태소만 남겨두기
test_1['ply_token']=test_1['ply_token'].map(lambda x: list(filter(lambda x: x[0] in unique_word, x)))
test_1.head(10)

test_1 data에서 형태소 분석한 결과를 적용시킨 data set

In [ ]:
test_1

test_2 data에서 앨범 최종 수정일 이후에 발매된 음악이 포함된 경우를 제외하는 과정

In [ ]:
test_2_song=test_2[['id', 'songs']]

filter2=test_2_song['songs'].astype('str')=='[]'
test_2_song_remove=test_2_song.loc[~filter2, :]

test_2_song_remove_unnest=np.dstack(
    (
        np.repeat(test_2_song_remove.id.values, list(map(len, test_2_song_remove.songs))),
        np.concatenate(test_2_song_remove.songs.values)
    )
  
)

test_2_song_remove=pd.DataFrame(data=test_2_song_remove_unnest[0], columns=test_2_song_remove.columns)
test_2_song_remove['id']=test_2_song_remove['id'].astype('str')
test_2_song_remove['songs']=test_2_song_remove['songs'].astype('str')

del test_2_song_remove_unnest

test_2_song_remove=test_2_song_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')

test_2_remove=test_2[~filter2]
test_2_remove['id']=test_2_remove['id'].astype('str')
test_2_remove=pd.merge(test_2_remove, test_2_song_remove)

test_2_remove

test_2_not_remove=test_2[filter2]

test_2_not_remove

test_2_remove['songs']=test_2_remove['remove_songs']
test_2_remove=test_2_remove.drop('remove_songs', axis=1)
test_2_fixed=pd.concat([test_2_remove, test_2_not_remove])
test_2_fixed

전처리된 test_1 data에 대해 word2vec 적용을 위해 필요한 data들만 추출한 dataframe 생성

In [ ]:
test_1_sent = test_1[['id', 'ply_token', 'tags']]
test_1_sent

전처리된 test_2 data에 대해 word2vec 적용을 위해 필요한 data들만 추출한 dataframe 생성

In [ ]:
test_2_sent = test_2_fixed[['id', 'songs']]
test_2_sent

In [ ]:
test_1_sent = pd.DataFrame(test_1_sent)
test_2_sent = pd.DataFrame(test_2_sent)


전처리된 test_1 data 중 플레이리스트 제목을 형태소 분해한 data에서 품사정보를 삭제

In [ ]:
#Word2vec 이용을 위해 ply_token data를 전처리
z = []
y = []
for i in range(len(test_1_sent['ply_token'])):
  z = []
  if len(test_1_sent['ply_token'][i]) == 0:
    x = test_1_sent['ply_token'][i]
    z = x
  else:
    for j in range(len(test_1_sent['ply_token'][i])):
     x = test_1_sent['ply_token'][i][j]
     z.insert(j,x[0])
  y.insert(i,z)
y

In [ ]:
test_1_sent['ply_token'] = y
test_1_sent # Word2vec 이용을 위해 필요한 data들만 모아둔 dataframe


In [ ]:
# test_1_sent에서 학습된 리스트에 존재하지 않는 ply_token을 제외함 (Word2vec 모델 적용을 위함)
for i in range(len(test_1_sent['ply_token'])):
  if len(test_1_sent['ply_token'][i]) == 0:
    test_1_sent['ply_token'][i] = test_1_sent['ply_token'][i]
  else:
    a = pd.DataFrame(test_1_sent['ply_token'][i], columns = ['token'])
    a['logical'] = a.isin(learning_list).astype(str)
    b = a['logical'] == 'True' 
    if b.sum() == 0:
      test_1_sent['ply_token'][i] = []
    else:
      c = c = a[b].drop('logical',axis=1)
      c = c.token.tolist()
      test_1_sent['ply_token'][i] = c


In [ ]:
# test_1_sent에서 학습된 리스트에 존재하지 않는 tags를 제외함 (Word2vec 모델 적용을 위함)
for i in range(len(test_1_sent['tags'])):
  if len(test_1_sent['tags'][i]) == 0:
    test_1_sent['tags'][i] = test_1_sent['tags'][i]
  else:
    a = pd.DataFrame(test_1_sent['tags'][i], columns = ['tag'])
    a['logical'] = a.isin(learning_list).astype(str)
    b = a['logical'] == 'True' 
    if b.sum() == 0:
      test_1_sent['tags'][i] = []
    else:
      c = c = a[b].drop('logical',axis=1)
      c = c.tag.tolist()
      test_1_sent['tags'][i] = c


In [ ]:
# test_2_sent에서 학습된 리스트에 존재하지 않는 songs를 제외함 (Word2vec 모델 적용을 위함)
for i in range(len(test_2_sent['songs'])):
  if len(test_2_sent['songs'][i]) == 0:
    test_2_sent['songs'][i] = test_2_sent['songs'][i]
  else:
    a = pd.DataFrame(test_2_sent['songs'][i], columns = ['song'])
    a['logical'] = a.isin(learning_list).astype(str)
    b = a['logical'] == 'True' 
    if b.sum() == 0:
      test_2_sent['songs'][i] = []
    else:
      c = c = a[b].drop('logical',axis=1)
      c = c.song.tolist()
      test_2_sent['songs'][i] = c


word2vec 적용을 위해 학습된 리스트에 존재하지 않는 data들을 제외하는 과정에서 input이 모두 사라져버린 data set들을 추출해내는 과정 -> Matrix Factorization 모델을 통해 곡 또는 태그를 채워서 다시 Word2vec 모델을 적용할 것

In [ ]:
m1 = []
for i in range(len(test_1_sent['id'])):
  if len(test_1_sent['ply_token'][i]) == 0 and len(test_1_sent['tags'][i]) == 0:
    m1.append(i)
  else:
    pass

test_1_sent_zerolen = test_1_sent.loc[m1]
test_1_sent_zerolen = test_1_sent_zerolen.reset_index(drop=True)
test_1_sent_zerolen # test_1_sent 중 전처리결과 token, tag, song 모두 하나도 남지 않게 된 플레이리스트

In [ ]:
m2 = []
for i in range(len(test_2_sent['id'])):
  if len(test_2_sent['songs'][i]) == 0:
    m2.append(i)
  else:
    pass

test_2_sent_zerolen = test_2_sent.loc[m2]
test_2_sent_zerolen = test_2_sent_zerolen.reset_index(drop=True)
test_2_sent_zerolen # test_2_sent 중 전처리결과 token, tag, song 모두 하나도 남지 않게 된 플레이리스트

Matrix Factorization 모델 적용을 위해 저장 후 전송

In [ ]:
test_1_sent_zerolen.to_json('빈태그 채우기.json')
test_2_sent_zerolen.to_json('빈노래 채우기.json')

기존 test 데이터셋에 제목과 태그가 존재했으나 Word2vec 이용을 위한 전처리 과정에서 제목과 태그가 사라져 Word2vec 모델 적용이 불가했던 데이터셋을 Matrix Factorization 모델을 통해 기존 데이터셋의 태그를 이용하여 예측태그를 형성 -> test_3_sent
기존 test 데이터셋에 곡 id가 존재했으나 Word2vec 이용을 위한 전처리 과정에서 곡 id가 사라져 Word2vec 모델 적용이 불가했던 데이터셋을 Matrix Factorization 모델을 통해 기존 데이터셋의 곡 id를 이용하여 예측 곡 id를 형성 -> test_4_sent


In [ ]:
test_3_sent = pd.read_json('빈태그 채우기.json')
test_4_sent = pd.read_json('빈노래 채우기.json')

In [ ]:
# test_3_sent에서 학습된 리스트에 존재하지 않는 tags를 제외함 (Word2vec 모델 적용을 위함)
for i in range(len(test_3_sent['predicted_tag'])):
    a = pd.DataFrame(test_3_sent['predicted_tag'][i], columns = ['predicted_tag'])
    a['logical'] = a.isin(learning_list).astype(str)
    b = a['logical'] == 'True' 
    if b.sum() == 0:
      test_3_sent['predicted_tag'][i] = []
    else:
      c = c = a[b].drop('logical',axis=1)
      c = c.predicted_tag.tolist()
      test_3_sent['predicted_tag'][i] = c

In [ ]:
# test_4_sent에서 학습된 리스트에 존재하지 않는 songs를 제외함 (Word2vec 모델 적용을 위함)
for i in range(len(test_4_sent['predicted_songs'])):
  a = pd.DataFrame(test_4_sent['predicted_songs'][i], columns = ['predicted_songs'])
  a['logical'] = a.isin(learning_list).astype(str)
  b = a['logical'] == 'True' 
  if b.sum() == 0:
    test_4_sent['predicted_songs'][i] = []
  else:
    c = c = a[b].drop('logical',axis=1)
    c = c.predicted_songs.tolist()
    test_4_sent['predicted_songs'][i] = c

In [ ]:
test_4_sent
length = test_4_sent.shape[0]

index = []
for i in range(length):
  if len(test_4_sent['predicted_songs'][i]) != 0:
    index.append(i)

index
# index = [] -> test_4_sent에는 learning list에 포함된 songs가 존재하지 않아서 Word2vec 모델 적용 불가능 -> empty case로 다룸

In [ ]:
#Word2vec 사용을 위해 플레이리스트 별로 태그를 한 문장으로 갖도록 test_3_sent_excluded를 전처리
w = []
for i in range(len(test_3_sent)):
  w.insert(i,test_3_sent['predicted_tag'][i])
test_3_input_excluded = w
test_3_input_excluded

test_3_input_expected data에 대해 Word2vec 모델을 이용하여 input과 유사한 output을 500개 뽑아내는 함수 정의

In [ ]:
def similar_1_list(x):
  aa = [wv_model.wv.most_similar(positive = x[i], topn = 500) for i in range(len(x))]
  return aa

In [ ]:
xxxx_3 = similar_1_list(test_3_input_excluded)


word2vec model을 적용한 결과에서 유사도 data를 제외하고 유사한 data만을 남기는 과정

In [ ]:
z = []
y_3 = []
for i in range(len(xxxx_3)):
  z = []
  for j in range(len(xxxx_3[i])):
    x5_3 = xxxx_3[i][j]
    z.insert(j,x5_3[0])
  y_3.insert(i,z)
y_3


test_3_sent에 predicted_songs column 생성

In [ ]:
predicted_songs = []
for i in range(len(test_3_sent)):
  predicted_songs.append([])

test_3_sent['predicted_songs'] = predicted_songs
test_3_sent


In [ ]:
shsh = list(list(filter(isinsonglist, y_3[k])) for k in range(len(y_3)))
for l in range(len(y_3)):
  test_3_sent['predicted_songs'][l] = shsh[l][:100]
  
test_3_sent
# 원래 test data set에서 제목과 일부 태그가 존재하는 플레이리스트였으나, 모델 적용을 위해 전처리했을 때 태그와 제목 형태소가 learning list에 속하지 않아 모델 적용이 불가능했던 data set.
# word2vec 모델 적용을 위해 matrix factorization method를 이용하여 일부 태그를 바탕으로 10개의 태그를 예측하였고, 10개의 예측 태그를 이용하여 word2vec 모델을 통해 100개의 예측 곡 id를 추출함.

In [ ]:
test_3_sent.to_json('MF+Word2vec으로 곡 예측.json')